# Premade Models

In the sections below we'll cover setting up your own model but most use cases will want a common set of spectra with a common set of properties. To avoid unnecessarily redefining common models every time Synthesizer is used we provide some premade models which can be used "out of the box" or as the foundation for constructing more complex models.

These premade models can be imported directly from the `emission_models` submodule which also defines a list containing all available models we have defined.

In [ ]:
from synthesizer.emission_models import (
    AGN_MODELS,
    COMMON_MODELS,
    PREMADE_MODELS,
    STELLAR_MODELS,
)

print(COMMON_MODELS)
print(STELLAR_MODELS)
print(AGN_MODELS)
print(PREMADE_MODELS)

To use any of these models the model simply needs importing and the required arguments passed, which arguments are required depends on the exact operation, more complex models further down the model "tree" will require more parameters to pass to the models which make them up.

Here is a demonstration of initialising different premade models with differing levels of complexity.

In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_models import (
    EmergentEmission,
    TotalEmission,
    TransmittedEmission,
)
from synthesizer.grid import Grid
from unyt import kelvin

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Get a simple extraction model (with optional fesc)
transmitted = TransmittedEmission(grid=grid, fesc=0.3)

print(transmitted)

# Get the emergent model which combines the escaped
# emission with an attenuated emission (here using incident
# for demonstration purposes) and requires some dust
# attenuation properties
emergent = EmergentEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=transmitted,
    tau_v=0.67,
    fesc=0.2,
)

print(emergent)

# Finally, define a Total emission model which
# is the combination of all other spectra
model = TotalEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
    fesc=0.2,
    fesc_ly_alpha=0.7,
    dust_emission_model=Blackbody(temperature=100 * kelvin),
)

print(model)

With these model now defined they can be passed into any emitter's spectra generation method but you can also visualise the different spectra and operations encapsulated by a model and see the "model tree". To do see we can use the `plot_emission_model` method.

Lets take a look at the total emission model.

In [ ]:
# Plot the model tree
model.plot_emission_tree()

Here we can see each individual spectra generated by the model and how they relate. Solid lines denote combinations of spectra into another, dashed lines denote an attenuation of the child spectra to produce the parent, blue boxes have no mask applied, green boxes (which we currently have none of) have a mask applied to them, rounded boxes are the result of a non-extraction operation, and square boxes are an extraction operation. You'll notice the extraction operations are always leaves in the tree.

Note that omitting `dust_emission_model` when initialising `TotalEmission` will result in a simpler model where `total == emergent`.

### Charlot and Fall

In addition to the simpler models shown above we also implement some more complex models. The first of these follows Charlot&Fall+2000:

- ...



In [ ]:
from synthesizer.emission_models import CharlotFall2000
from unyt import dimensionless

# Either with dust emission...
cf_model = CharlotFall2000(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-0.7),
    dust_curve_nebular=PowerLaw(slope=-1.3),
    age_pivot=7 * dimensionless,
    dust_emission_ism=Blackbody(temperature=50 * kelvin),
    dust_emission_nebular=Blackbody(temperature=100 * kelvin),
)

cf_model.plot_emission_tree(fontsize=6)

# Or without
cf_model = CharlotFall2000(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-0.7),
    dust_curve_nebular=PowerLaw(slope=-1.3),
    age_pivot=7 * dimensionless,
)

cf_model.plot_emission_tree()

It's also possible to plot a subtree within a model by passing the root of the subtree. This is particularly helpful for models like `CharlotFall2000` which define a lot of extra spectra that don't necessarily appear in the main tree depending on the exact inputs. You can see this in the plots below which show the trees defining the "extra" spectra available when using `CharlotFall2000`.

In [ ]:
cf_model.plot_emission_tree(root="incident")

In [ ]:
cf_model.plot_emission_tree(root="transmitted")

In [ ]:
cf_model.plot_emission_tree(root="intrinsic")

In [ ]:
cf_model.plot_emission_tree(root="attenuated")

### Pacman

We also implement a more generalised form of Charlot&Fall+2000 which we call "Pacman":

- ...

In [ ]:
from synthesizer.emission_models import BimodalPacmanEmission, PacmanEmission

# Simple Pacman with dust emission
simple_pc_model = PacmanEmission(
    grid=grid,
    tau_v=0.7,
    dust_curve=PowerLaw(slope=-1.3),
    dust_emission=Blackbody(temperature=100 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)

simple_pc_model.plot_emission_tree()

# ... or without
simple_pc_model = PacmanEmission(
    grid=grid,
    tau_v=0.7,
    dust_curve=PowerLaw(slope=-1.3),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)

simple_pc_model.plot_emission_tree()

# Pacman split by age
pc_model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_nebular=PowerLaw(slope=-0.7),
    dust_emission_ism=Blackbody(temperature=100 * kelvin),
    dust_emission_nebular=Blackbody(temperature=30 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)

pc_model.plot_emission_tree(fontsize=5)

# ... or without
pc_model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_nebular=PowerLaw(slope=-0.7),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)

pc_model.plot_emission_tree(fontsize=6)